<a href="https://colab.research.google.com/github/MohneetKaur/NLP-using-GoogleColab/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [85]:
import pandas as pd

In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
df=pd.read_csv('/content/drive/MyDrive/train.csv')

In [113]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [114]:
df.shape

(20800, 5)

In [115]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [116]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [117]:
###Drop Nan Values
df=df.dropna()


In [118]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [119]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [120]:
## Get the Dependent features
y=df['label']

In [121]:
X.shape

(18285, 4)

In [122]:
y.shape

(18285,)

In [123]:
import tensorflow as tf

In [124]:
tf.__version__

'2.12.0'

In [125]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [126]:
### Vocabulary size - no. of unique words
voc_size=5000

### Onehot Representation

In [127]:
messages=X.copy()

In [128]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [129]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [135]:
messages.reset_index(inplace=True)

In [136]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [137]:
import nltk
import re
from nltk.corpus import stopwords

In [138]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [139]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [140]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [141]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [142]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] # returns index in a list
onehot_repr

[[3025, 2533, 2279, 1446, 405, 4067, 1114, 281, 4489, 1769],
 [932, 345, 1608, 4146, 3834, 707, 2508],
 [4341, 3623, 4164, 3972],
 [2686, 2787, 3613, 4554, 3924, 293],
 [4313, 3834, 1911, 4001, 2708, 215, 3834, 2747, 3766, 4028],
 [1359,
  2683,
  3085,
  2081,
  3438,
  3785,
  3754,
  1886,
  4944,
  3640,
  2998,
  4151,
  1814,
  3291,
  2508],
 [535, 3162, 1832, 4570, 108, 3346, 3856, 3789, 4804, 2438, 948],
 [2734, 347, 3254, 4107, 2624, 644, 3785, 1673, 4804, 2438, 948],
 [4567, 1976, 2782, 4821, 4452, 1973, 4229, 221, 3785, 3702],
 [4083, 4745, 2014, 4849, 4551, 1217, 3593, 1784],
 [4517, 3768, 1381, 2900, 4075, 1228, 1916, 4353, 705, 2450, 2907],
 [4554, 4, 405, 1973, 3785, 2624],
 [2186, 926, 2271, 817, 1853, 1332, 1398, 4849, 2104],
 [1754, 1476, 4772, 3648, 1624, 993, 2626, 4804, 2438, 948],
 [4618, 4234, 4626, 4908, 2889, 4804, 2438, 948],
 [346, 3225, 2863, 1149, 1231, 1395, 1773, 3860, 1850, 3125],
 [446, 3783, 345],
 [3251, 4563, 2589, 1161, 3785, 3234, 1573, 2508],
 [3

In [143]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [144]:
onehot_repr[1]

[932, 345, 1608, 4146, 3834, 707, 2508]

### Embedding Representation

In [145]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3025 2533 2279 ...    0    0    0]
 [ 932  345 1608 ...    0    0    0]
 [4341 3623 4164 ...    0    0    0]
 ...
 [1698 4602 3338 ...    0    0    0]
 [1221 2624 3363 ...    0    0    0]
 [2638 3839 2444 ...    0    0    0]]


In [146]:
embedded_docs[1]

array([ 932,  345, 1608, 4146, 3834,  707, 2508,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [147]:
embedded_docs[0]

array([3025, 2533, 2279, 1446,  405, 4067, 1114,  281, 4489, 1769,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [148]:
## Creating model
embedding_vector_features=40 ##features representation - each and every word will be represented by vector of 40 size
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)) # 100 neurons
model.add(Dense(1,activation='sigmoid')) # 1- becuase binary - fake/not fake
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               56400     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [149]:
len(embedded_docs),y.shape

(18285, (18285,))

In [150]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [151]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [152]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [153]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 52ms/step - loss: 0.3034 - accuracy: 0.8553 - val_loss: 0.2291 - val_accuracy: 0.9044
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1321 - accuracy: 0.9488 - val_loss: 0.2297 - val_accuracy: 0.9158
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0832 - accuracy: 0.9713 - val_loss: 0.2816 - val_accuracy: 0.9155
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0601 - accuracy: 0.9804 - val_loss: 0.3262 - val_accuracy: 0.9152
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0393 - accuracy: 0.9874 - val_loss: 0.3146 - val_accuracy: 0.9084
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0304 - accuracy: 0.9898 - val_loss: 0.4043 - val_accuracy: 0.9118
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0266 - accuracy: 0.9920 - val_loss: 0.4004 - val_accuracy: 0.9100


### Adding Dropout

In [154]:
from tensorflow.keras.layers import Dropout
## Adding dropout layer
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [155]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 60ms/step - loss: 0.3219 - accuracy: 0.8442 - val_loss: 0.2077 - val_accuracy: 0.9117
Epoch 2/10
192/192 [==============================] - 3s 16ms/step - loss: 0.1476 - accuracy: 0.9449 - val_loss: 0.1999 - val_accuracy: 0.9203
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1074 - accuracy: 0.9619 - val_loss: 0.2247 - val_accuracy: 0.9112
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0819 - accuracy: 0.9699 - val_loss: 0.2305 - val_accuracy: 0.9163
Epoch 5/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0642 - accuracy: 0.9788 - val_loss: 0.2644 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0484 - accuracy: 0.9838 - val_loss: 0.3213 - val_accuracy: 0.9138
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0438 - accuracy: 0.9851 - val_loss: 0.3358 - val_accuracy: 0.9107

### Performance Metrics And Accuracy

In [156]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [157]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [158]:
from sklearn.metrics import confusion_matrix

In [159]:
confusion_matrix(y_test,y_pred)

array([[3191,  228],
       [ 352, 2264]])

In [160]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.903893951946976

In [161]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.93      0.92      3419
           1       0.91      0.87      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

